In [1]:
import pandas as pd

url = 'D:\Google Drive\Deep_Learning\Quantium_Intership'

data = pd.read_csv(url+'\QVI_data.csv')

#trdata['DATE'] = pd.to_datetime(trdata['DATE'], origin ='1899/12/30', unit = 'D')



In [70]:
data.head()

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream


In [2]:
ds = data.copy()

ds['DATE'] = pd.to_datetime(ds['DATE'], errors='coerce')
ds['DATE_YM'] = ds['DATE'].dt.strftime('%Y%m')
#ds['DATE_YM'] = pd.to_datetime(ds['DATE_YM'], errors='coerce')
ds.dtypes

LYLTY_CARD_NBR               int64
DATE                datetime64[ns]
STORE_NBR                    int64
TXN_ID                       int64
PROD_NBR                     int64
PROD_NAME                   object
PROD_QTY                     int64
TOT_SALES                  float64
PACK_SIZE                    int64
BRAND                       object
LIFESTAGE                   object
PREMIUM_CUSTOMER            object
DATE_YM                     object
dtype: object

In [3]:
mask = (ds['DATE_YM'] > '201901') & (ds['DATE_YM'] <= '201905')

ds3 = ds.loc[mask]
ds1 = ds

In [237]:
ds1[ds1['STORE_NBR']==77][['TOT_SALES','LYLTY_CARD_NBR','PROD_QTY']]

,TOT_SALES,LYLTY_CARD_NBR,PROD_QTY
73365,3.3,77000,1
73366,3.0,77000,1
73367,10.8,77000,2
73368,11.4,77001,2
73369,9.2,77001,2
...,...,...,...
264818,8.6,2330321,2
264819,3.4,2330331,2
264820,4.4,2330431,1
264821,3.8,2330461,1


In [269]:
ds1.groupby('STORE_NBR')[['TOT_SALES','LYLTY_CARD_NBR','PROD_QTY']].count().sort_values(['TOT_SALES','LYLTY_CARD_NBR','PROD_QTY'],ascending=False).head(15)

,TOT_SALES,LYLTY_CARD_NBR,PROD_QTY
STORE_NBR,,,
226,2020,2020,2020
88,1873,1873,1873
93,1832,1832,1832
165,1819,1819,1819
237,1785,1785,1785
43,1771,1771,1771
40,1764,1764,1764
230,1751,1751,1751
156,1747,1747,1747


In [69]:
def f(x):
    d = {}
    d['Tot_Sales'] = x['TOT_SALES'].sum()
    d['nCustomers'] = x['LYLTY_CARD_NBR'].nunique()
    d['nTxnPerCust'] = (x['TXN_ID'].count()/ x['LYLTY_CARD_NBR'].nunique()).sum()
    d['nChipsPerTxn'] = (x['PROD_QTY'] / x['TXN_ID'].count()).sum()
    d['avgPricePerUnit'] = (x['TOT_SALES'] / x['PROD_QTY']).mean()
    return pd.Series(d, index=['Tot_Sales', 'nCustomers', 'nTxnPerCust', 'nChipsPerTxn','avgPricePerUnit'])

#.sort_values('Tot_Sales',ascending=False)

measureOverTime = ds1.groupby(['STORE_NBR','DATE_YM'],as_index=True).apply(f).sort_values('Tot_Sales',ascending=False)
measureOverTime.reset_index(level=0, inplace=True)
measureOverTime.reset_index(level=0, inplace=True)
measureOverTime

,DATE_YM,STORE_NBR,Tot_Sales,nCustomers,nTxnPerCust,nChipsPerTxn,avgPricePerUnit
0,201812,226,1659.6,150.0,1.260000,1.994709,4.402646
1,201807,58,1627.2,146.0,1.273973,1.994624,4.386022
2,201811,226,1602.6,141.0,1.312057,2.000000,4.331351
3,201903,226,1561.6,133.0,1.270677,2.000000,4.620118
4,201905,226,1530.9,140.0,1.292857,1.883978,4.497790
...,...,...,...,...,...,...,...
3164,201807,211,2.6,1.0,1.000000,1.000000,2.600000
3165,201807,117,2.6,1.0,1.000000,1.000000,2.600000
3166,201904,211,2.6,1.0,1.000000,1.000000,2.600000
3167,201901,193,1.8,1.0,1.000000,1.000000,1.800000


In [70]:
storesWithFullObs = measureOverTime.groupby('STORE_NBR').mean()
storesWithFullObs.reset_index(level=0, inplace=True)
storesWithFullObs

,STORE_NBR,Tot_Sales,nCustomers,nTxnPerCust,nChipsPerTxn,avgPricePerUnit
0,1,199.466667,45.333333,1.052572,1.229720,3.431835
1,2,167.150000,39.916667,1.057976,1.114382,3.544056
2,3,1066.870833,106.000000,1.180848,1.940044,4.413802
3,4,1220.637500,116.416667,1.199132,1.975358,4.411780
4,5,791.733333,91.000000,1.246227,1.986207,3.520065
...,...,...,...,...,...,...
267,268,216.754167,43.250000,1.051862,1.340092,3.524668
268,269,935.150000,104.666667,1.279210,1.985402,3.513618
269,270,941.162500,102.833333,1.322187,1.985812,3.489192
270,271,810.150000,92.333333,1.250214,1.978626,3.546851


In [71]:
preTrialMeasures = measureOverTime[measureOverTime['DATE_YM'] < '201902'].sort_values('DATE_YM',ascending=True)

preTrialMeasures = preTrialMeasures.groupby('STORE_NBR').mean()
preTrialMeasures.reset_index(level=0, inplace=True)
preTrialMeasures

,STORE_NBR,Tot_Sales,nCustomers,nTxnPerCust,nChipsPerTxn,avgPricePerUnit
0,1,198.128571,45.285714,1.046852,1.219841,3.445854
1,2,161.214286,38.857143,1.055467,1.113880,3.483357
2,3,1075.164286,106.285714,1.179460,1.946726,4.418498
3,4,1303.857143,121.285714,1.229716,1.986011,4.397765
4,5,819.957143,93.000000,1.257539,1.985969,3.542777
...,...,...,...,...,...,...
266,268,221.292857,43.428571,1.053291,1.376889,3.462278
267,269,952.071429,106.571429,1.281280,1.983766,3.511171
268,270,956.850000,104.857143,1.330598,1.982375,3.464519
269,271,823.585714,93.142857,1.252544,1.985856,3.547003


In [72]:

def fun(x):
    x['Rank1'] = x['Tot_Sales'].rank()
    x['Rank2'] = x['nCustomers'].rank()
    x['Rank3'] = x['nTxnPerCust'].rank()
    x['Rank4'] = x['nChipsPerTxn'].rank()
    x['Rank5'] = x['avgPricePerUnit'].rank()
    x['RankTotal'] = x['Rank1']+x['Rank2']+x['Rank3']+x['Rank4']+x['Rank5']
    return x

ctrl = fun(storesWithFullObs).sort_values('RankTotal',ascending=False).head(10)
ctrl


,STORE_NBR,Tot_Sales,nCustomers,nTxnPerCust,nChipsPerTxn,avgPricePerUnit,Rank1,Rank2,Rank3,Rank4,Rank5,RankTotal
216,217,1166.133333,109.416667,1.209573,1.993082,4.423381,257.0,241.5,179.0,264.0,237.0,1178.5
39,40,1296.625000,125.250000,1.173924,1.984433,4.450833,269.0,270.0,159.0,214.0,265.0,1177.0
25,26,1205.775000,115.583333,1.184243,1.984684,4.447637,263.0,260.0,165.0,217.0,262.0,1167.0
164,165,1331.145833,124.000000,1.222377,1.981602,4.436387,270.0,268.0,181.0,192.0,253.0,1164.0
87,88,1361.104167,126.250000,1.236596,1.985232,4.393104,271.0,271.0,185.0,225.0,199.0,1151.0
42,43,1032.045833,111.583333,1.321488,1.987879,3.525446,236.0,251.0,266.0,248.0,145.0,1146.0
57,58,1270.954167,120.916667,1.198701,1.985398,4.405612,267.0,267.0,174.0,227.0,210.0,1145.0
92,93,1067.816667,116.166667,1.315341,1.986680,3.520450,240.0,262.0,265.0,240.0,137.0,1144.0
198,199,1233.083333,116.916667,1.202840,1.981781,4.424745,266.0,265.0,178.0,194.0,240.0,1143.0
229,230,1023.916667,111.166667,1.313032,1.986252,3.536504,234.0,250.0,264.0,237.0,152.0,1137.0


In [73]:
pre = fun(preTrialMeasures).sort_values('RankTotal',ascending=False).head(10)
pre

,STORE_NBR,Tot_Sales,nCustomers,nTxnPerCust,nChipsPerTxn,avgPricePerUnit,Rank1,Rank2,Rank3,Rank4,Rank5,RankTotal
163,165,1319.685714,123.142857,1.216393,1.991469,4.430821,268.0,266.5,177.0,246.0,246.0,1203.5
215,217,1187.328571,109.428571,1.224498,1.996544,4.439239,259.0,236.0,180.0,260.0,255.0,1190.0
39,40,1266.685714,123.142857,1.167189,1.991718,4.429626,264.0,266.5,154.0,247.0,244.0,1175.5
201,203,1277.671429,120.285714,1.209177,1.985429,4.431855,265.0,264.0,172.0,203.0,248.0,1152.0
42,43,1034.157143,111.142857,1.317889,1.990602,3.553940,234.0,245.0,263.0,241.0,162.0,1145.0
25,26,1209.057143,116.857143,1.170988,1.984733,4.457358,261.0,261.0,159.0,198.0,263.0,1142.0
3,4,1303.857143,121.285714,1.229716,1.986011,4.397765,266.0,265.0,181.0,209.0,210.0,1131.0
211,213,1027.028571,112.428571,1.291477,1.991001,3.547475,233.0,248.5,243.0,243.0,157.0,1124.5
92,94,995.992857,108.000000,1.300438,1.998311,3.535080,228.0,231.0,250.0,263.0,149.0,1121.0
71,72,1216.928571,114.428571,1.215458,1.986401,4.403292,262.0,254.0,176.0,214.0,214.0,1120.0


In [98]:
preTrialMeasures.corr()

,STORE_NBR,Tot_Sales,nCustomers,nTxnPerCust,nChipsPerTxn,avgPricePerUnit,Rank1,Rank2,Rank3,Rank4,Rank5,RankTotal
STORE_NBR,1.000000,-0.032903,-0.040845,-0.020683,-0.031588,-0.014020,-0.026651,-0.023601,-0.012004,-0.066640,-0.014457,-0.035745
Tot_Sales,-0.032903,1.000000,0.980149,0.847043,0.702704,0.332461,0.986988,0.968870,0.837110,0.693894,0.329924,0.951620
nCustomers,-0.040845,0.980149,1.000000,0.884912,0.653267,0.233327,0.958870,0.967820,0.876517,0.670880,0.242353,0.926593
nTxnPerCust,-0.020683,0.847043,0.884912,1.000000,0.623327,-0.083780,0.798729,0.806657,0.978357,0.755618,-0.032056,0.824560
nChipsPerTxn,-0.031588,0.702704,0.653267,0.623327,1.000000,0.384779,0.726551,0.660294,0.645124,0.834009,0.357042,0.803589
avgPricePerUnit,-0.014020,0.332461,0.233327,-0.083780,0.384779,1.000000,0.411682,0.287877,-0.001655,0.127513,0.887990,0.427248
Rank1,-0.026651,0.986988,0.958870,0.798729,0.726551,0.411682,1.000000,0.974620,0.805621,0.679344,0.400877,0.962516
Rank2,-0.023601,0.968870,0.967820,0.806657,0.660294,0.287877,0.974620,1.000000,0.804942,0.639503,0.285865,0.923733
Rank3,-0.012004,0.837110,0.876517,0.978357,0.645124,-0.001655,0.805621,0.804942,1.000000,0.750336,0.038179,0.847441
Rank4,-0.066640,0.693894,0.670880,0.755618,0.834009,0.127513,0.679344,0.639503,0.750336,1.000000,0.164004,0.806110


In [93]:
ctrl.query('STORE_NBR == "77"|STORE_NBR == "86"|STORE_NBR == "88"')

,STORE_NBR,Tot_Sales,nCustomers,nTxnPerCust,nChipsPerTxn,avgPricePerUnit,Rank1,Rank2,Rank3,Rank4,Rank5,RankTotal
87,88,1361.104167,126.25,1.236596,1.985232,4.393104,271.0,271.0,185.0,225.0,199.0,1151.0


In [51]:

#storesWithFullObs[['Tot_Sales','nCustomers']].corr()
storesWithFullObs = storesWithFullObs[storesWithFullObs['STORE_NBR'].notnull()]

for i in storesWithFullObs['STORE_NBR']:
    calculatedMeasure = (storesWithFullObs.loc[i,'Tot_Sales']-storesWithFullObs.loc[10,'Tot_Sales'])/storesWithFullObs.loc[10,'Tot_Sales']
    




KeyError: True

In [43]:
measureOverTime[['Rank1','Rank2','Rank3','Rank4','Rank5']].corr()

,Rank1,Rank2,Rank3,Rank4,Rank5
Rank1,1.000000,0.359494,0.847092,0.672012,0.347063
Rank2,0.359494,1.000000,-0.040595,-0.178713,0.489995
Rank3,0.847092,-0.040595,1.000000,0.804596,0.046244
Rank4,0.672012,-0.178713,0.804596,1.000000,0.086704
Rank5,0.347063,0.489995,0.046244,0.086704,1.000000


DATE_YM              12
STORE_NBR           272
Tot_Sales          2904
nCustomers          130
nTxnPerCust         954
nChipsPerTxn       1196
avgPricePerUnit    2991
dtype: int64

In [254]:
preTrialMeasures = measureOverTime[measureOverTime['DATE_YM'] < '201902']
preTrialMeasures.head()

,DATE_YM,STORE_NBR,Tot_Sales,nCustomers,nTxnPerCust,nChipsPerTxn,avgPricePerUnit
0,201812,226,1659.6,150.0,1.260000,1.994709,4.402646
1,201807,58,1627.2,146.0,1.273973,1.994624,4.386022
2,201811,226,1602.6,141.0,1.312057,2.000000,4.331351
6,201901,4,1525.0,134.0,1.253731,1.994048,4.549405
7,201810,226,1519.0,139.0,1.251799,2.000000,4.364943


In [261]:
ds1[ds1['STORE_NBR']==77].groupby(['STORE_NBR','DATE_YM']).apply(f).sort_values('Tot_Sales',ascending=False).head()

Tot_Sales  nCustomers  nTxnPerCust  nChipsPerTxn  \
STORE_NBR DATE_YM                                                     
77        201905       299.3        55.0     1.018182      1.500000   
          201807       296.8        51.0     1.078431      1.527273   
          201903       278.5        50.0     1.100000      1.490909   
          201812       267.3        46.0     1.065217      1.469388   
          201906       264.7        41.0     1.024390      1.666667   

                   avgPricePerUnit  
STORE_NBR DATE_YM                   
77        201905          3.528571  
          201807          3.545455  
          201903          3.481818  
          201812          3.708163  
          201906          3.735714